# Multi-Agent System

In [1]:
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent, ReActAgent, ToolCallResult, AgentStream  
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
import chromadb


# define some tools
def add(a: int, b: int) -> int:
    return a + b


def subtract(a: int, b: int) -> int:
    return a - b


# for HuggingFace                                                                                          
# from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI                                     
# from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding                      
# embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")                      
# llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")                              

# create a vector store                                                                                    
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# create a query engine                                                                                    
llm = Ollama(model="qwen2.5-coder")
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# create agent configs
calculator_agent = FunctionAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

# create and run the workflow
agent = AgentWorkflow(
    agents=[calculator_agent, query_agent], root_agent="calculator"
)

# run the system
handler = agent.run(user_msg="Can you add 5 and 3?")

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  add {'a': 5, 'b': 3} => 8
The sum of 5 and 3 is 8. Is there anything else I can help with?

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='The sum of 5 and 3 is 8. Is there anything else I can help with?')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='add', tool_output=ToolOutput(content='8', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], raw={'model': 'qwen2.5-coder', 'created_at': '2025-03-21T00:26:50.178665Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1047951583, 'load_duration': 17323125, 'prompt_eval_count': 410, 'prompt_eval_duration': 206283416, 'eval_count': 22, 'eval_duration': 819119042, 'message': Message(role='assistant', content='The sum of 5 and 3 is 8. Is there anything else I can help with?', images=None, tool_calls=None), 'usage': {'prompt_tokens': 410, 'completion_tokens': 22, 'total_tokens': 432}}, current_agent_name='calc